# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 1 2022 4:26PM,249836,19,TR-RG-11022022,"GCH Granules USA, Inc.",Released
1,Nov 1 2022 3:54PM,249831,19,TR-RN-11022022BLK,"GCH Granules USA, Inc.",Released
2,Nov 1 2022 3:28PM,249828,10,8720390,Livs Products,Released
3,Nov 1 2022 3:25PM,249823,10,0086134506,ISDIN Corporation,Executing
4,Nov 1 2022 3:25PM,249823,10,0086134511,ISDIN Corporation,Executing
5,Nov 1 2022 3:25PM,249823,10,0086134509,ISDIN Corporation,Executing
6,Nov 1 2022 3:25PM,249823,10,0086134513,ISDIN Corporation,Executing
7,Nov 1 2022 3:25PM,249823,10,0086134510,ISDIN Corporation,Executing
8,Nov 1 2022 3:25PM,249823,10,0086134512,ISDIN Corporation,Executing
9,Nov 1 2022 3:25PM,249823,10,0086134507,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,249825,Released,1
7,249826,Released,2
8,249828,Released,1
9,249831,Released,1
10,249836,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249825,NaN,1.0
249826,NaN,2.0
249828,NaN,1.0
249831,NaN,1.0
249836,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249793,1.0,0.0
249794,0.0,1.0
249795,0.0,1.0
249807,0.0,15.0
249823,6.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249793,1,0
249794,0,1
249795,0,1
249807,0,15
249823,6,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249793,1,0
1,249794,0,1
2,249795,0,1
3,249807,0,15
4,249823,6,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249793,1,
1,249794,,1
2,249795,,1
3,249807,,15
4,249823,6,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc."
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc."
2,Nov 1 2022 3:28PM,249828,10,Livs Products
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation
27,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc."
29,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc."
30,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd"
45,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc."
46,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc."
47,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc.",,1
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc.",,1
2,Nov 1 2022 3:28PM,249828,10,Livs Products,,1
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,6,18
4,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",,2
5,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",,1
6,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd",,15
7,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",,1
8,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",,1
9,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc.",1,
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc.",1,
2,Nov 1 2022 3:28PM,249828,10,Livs Products,1,
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,18,6
4,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,
5,Nov 1 2022 3:20PM,249825,22,"NBTY Global, Inc.",1,
6,Nov 1 2022 1:39PM,249807,19,"AdvaGen Pharma, Ltd",15,
7,Nov 1 2022 12:05PM,249795,21,"NBTY Global, Inc.",1,
8,Nov 1 2022 12:05PM,249794,21,"NBTY Global, Inc.",1,
9,Nov 1 2022 12:04PM,249793,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc.",1,
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc.",1,
2,Nov 1 2022 3:28PM,249828,10,Livs Products,1,
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,18,6
4,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc.",1.0,NaN
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc.",1.0,NaN
2,Nov 1 2022 3:28PM,249828,10,Livs Products,1.0,NaN
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,18.0,6.0
4,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 1 2022 4:26PM,249836,19,"GCH Granules USA, Inc.",1.0,0.0
1,Nov 1 2022 3:54PM,249831,19,"GCH Granules USA, Inc.",1.0,0.0
2,Nov 1 2022 3:28PM,249828,10,Livs Products,1.0,0.0
3,Nov 1 2022 3:25PM,249823,10,ISDIN Corporation,18.0,6.0
4,Nov 1 2022 3:21PM,249826,19,"Methapharm, Inc.",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,499651,19.0,6.0
19,999300,19.0,0.0
21,749382,2.0,1.0
22,249825,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,499651,19.0,6.0
1,19,999300,19.0,0.0
2,21,749382,2.0,1.0
3,22,249825,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,19.0,6.0
1,19,19.0,0.0
2,21,2.0,1.0
3,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,19,Released,19.0
2,21,Released,2.0
3,22,Released,1.0
4,10,Executing,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,21,22
Status,,,,
Executing,6.0,0.0,1.0,0.0
Released,19.0,19.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,21,22
0,Executing,6.0,0.0,1.0,0.0
1,Released,19.0,19.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,21,22
0,Executing,6.0,0.0,1.0,0.0
1,Released,19.0,19.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()